In [1]:
data_partition = {
        "gwilliams2023": {
            "testing_subjects": [19, 20, 21],
            "testing_tasks": [0],
        }
        # "schoffelen2022": {
        #     "testing_subjects": [],
        #     "testing_tasks": [8, 9],
        # },
}

In [2]:
# from utils.compression import compress_directories, decompress_directories

# for base_path, batch_type in data_partition.items():
#     decompress_directories(
#         source_path=f'downloaded_data/{base_path}',
#         destination_path=f'data/{base_path}',
#         checksum_file_name="checksums.txt",
#         delete_compressed_files=True,
#         num_workers=None
#     )

In [2]:
from config import SimpleConvConfig
from models.simpleconv import SimpleConv
import torch

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=True,
    conditions=None,
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=256,
    dropout=0.2,
    initial_batch_norm=False,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=None,
    merger_channels=None,
    merger_dropout=None,
    merger_conditional=None,
    # Inital
    initial_linear=False,
    initial_depth=0,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=4,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=False,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

In [3]:
from sympy import false
from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
import multiprocessing

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=100,
    frequency_bands={'all': (None, None)},
    max_random_shift=1.0,
    window_size= 4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=None,
    notch_filter=True,
    scaling='both',
    # Hyperparameters
    learning_rate = 3e-4,
    weight_decay = 1e-4,
    epochs = 50,
    batch_size = 256,
    use_clip_loss = True,
    use_mse_loss = True,
    alpha =0.6,
    random_test_size =10,
    seed = 42,
)

session = TrainingSessionV0(
    config=config,
    studies={"gwilliams2023": 'audio'},
    data_path='data',
    save_path='saves/Dec17_CLIP_MSE_baseline',
    clear_cache=False,
)

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

SimpleConv initialized with 2524704 parameters, cond: []
Merger False, merger channels None
ConvBlocks: 4, hidden_dim: 256, params 2327040


In [ ]:
try:
    session.train(
        device='cuda',
        buffer_size=30,
        num_workers=multiprocessing.cpu_count() - 2,
        max_cache_size=200,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print('Exited')

GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.


2024-12-18 16:22:10,923	INFO worker.py:1821 -- Started a local Ray instance.
Training Epoch 1: 100%|██████████| 135/135 [02:24<00:00,  1.07s/it]


Testing at epoch 1
Test unseen_subject completed. Accuracy: 0.0055, Top 1: 0.0084, Top 5: 0.0582, Top 10: 0.1117, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.0091, Top 1: 0.0142, Top 5: 0.0727, Top 10: 0.1447, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0049, Top 1: 0.0099, Top 5: 0.0605, Top 10: 0.1310, Perplexity: 0.0000
Epoch 1 completed in 2.70m. 0.02m per recording.


Training Epoch 2:  15%|█▍        | 20/135 [00:38<02:03,  1.07s/it]